# Analyzing Subjects:
<break> </break>
<font size=4>
The Analyzer, along with the classifier class, provide lots of functionality towards analyzing the Cool Neighbors classification data.
</font>

## Aggregating the Classifications

The Analyzer class object requires the extraction and reduction files produced via the Aggregator class. See the Aggregating example for more details.

In [1]:
import os
from DataToolkit.Aggregator import Aggregator
# For more details about Aggregating, see the Aggregating example.

# This is the workflow ID and version for the Backyard Worlds: Cool Neighbors project's Launch-0 workflow.
workflow_id = 24299
version = 1.6

# Default names for the CSV files that are exported via Zooniverse's data exports tab.
classifications_csv = "backyard-worlds-cool-neighbors-classifications.csv"
workflows_csv = "backyard-worlds-cool-neighbors-workflows.csv"
config_directory = "Config"
extractions_directory = "Extractions"
reductions_directory = "Reductions"

# Check whether the aggregated files already exist for this workflow and version
aggregator = Aggregator(classifications_csv_filename=classifications_csv, workflow_csv_filename=workflows_csv, config_directory=config_directory, extractions_directory=extractions_directory, reductions_directory=reductions_directory)

if(os.path.exists("{}/question_extractor_workflow_{}_V{}.csv".format(aggregator.extractions_directory, workflow_id, version)) and os.path.exists("{}/question_reducer_workflow_{}_V{}.csv".format(aggregator.reductions_directory, workflow_id, version))):
    print("Aggregated files already exist, skipping aggregation.")
else:
    aggregator.aggregateWorkflow(workflow_id=workflow_id, v=version)

Command:  panoptes_aggregation config C:\Users\Austin\Documents\GitHub\DataToolkit\DataToolkit\Examples\JupyterNotebooks\Analyzing\backyard-worlds-cool-neighbors-workflows.csv 24299 -d C:\Users\Austin\Documents\GitHub\DataToolkit\DataToolkit\Examples\JupyterNotebooks\Analyzing\Config -v 1.6


FileNotFoundError: [WinError 2] The system cannot find the file specified

## Getting Subjects from Zooniverse

In order to access subjects, once you've logged in, you can either request all subjects from a particular project, from a particular subject set, or you can request subjects directly via their subject id.

In [ ]:
# Provide the project id and subject set id you wish to get subjects from.
# By default, the project ID and subject set ID are saved to the filenames "project_id.pickle" and "subject_set_id.pickle" 
# and save is set to True.
project_id, subject_set_id = Spout.requestZooniverseIDs(filenames=["project_id.pickle", "subject_set_id.pickle"], save=True)

print("Warning: This will take a long time to run if there are a lot of subjects in the project.")

# Get all subjects in the project.
project_subjects = Spout.get_subjects_from_project(project_id, only_orphans=False)
print(f"Number of subjects in the project: {len(project_subjects)}\n")

# Get all orphaned subjects in the project.
orphaned_project_subjects = Spout.get_subjects_from_project(project_id, only_orphans=True)
print(f"Number of orphaned subjects in the project: {len(orphaned_project_subjects)}\n")

# Get all subjects in the subject set.
subject_set_subjects = Spout.get_subjects_from_project(project_id, subject_set_id, only_orphans=False)
print(f"Number of subjects in the subject set: {len(subject_set_subjects)}\n")

# Get a single subject by its ID.
subject_id = None
if(subject_id is not None):
    single_subject = Spout.get_subject(subject_id)
    print(f"Single subject: {single_subject}\n")

# With the list of subjects, or individual subjects, you can do whatever you want with them. For example, you can find a subset of
# subjects that meet a certain criteria using the SubjectDiscriminator class (see DataToolkit\Discriminator.py).

# Additionally, you can get a user object from their ID or username:
user_id = "austinh2001"
if(user_id is not None):
    user = Spout.get_user(user_id)
    print(f"User: {user}")

## Modifying Subjects

In order to avoid accidental modifications to subjects, the modification functions will be commented out. 
If you would like to use them, please uncomment them. Be sure to verify you are modifying the correct subjects before uncommenting them.

In [ ]:
subjects = []

# Remove subjects from a subject set:
"""
Spout.remove_subjects(project_id, subject_set_id, subjects)
"""

# Delete subjects from Zooniverse:
"""
Spout.delete_subjects(subjects)
"""

# Modify subject metadata field names:
"""
Spout.modify_subject_metadata_field_name(subjects, "Ecliptic Coordinates", "#Ecliptic Coordinates")
"""

# Modify subject metadata field values:
"""
Spout.modify_subject_metadata_field_value(subjects, "FOV", "~120.0 x ~120.0 arcseconds")
"""

# Check if a subject has images:
"""
for subject in subjects:
    print(f"Subject {subject.id} has images: {Spout.subject_has_images(subject)}")
"""

# Check if a subject has metadata:
"""
for subject in subjects:
    print(f"Subject {subject.id} has metadata: {Spout.subject_has_metadata(subject)}")
"""